In [8]:
import pandas as pd
import numpy as np

In [9]:
test_ad=pd.read_csv("../test/ad.csv")
test_click_log=pd.read_csv("../test/click_log.csv")

In [10]:
train_ad=pd.read_csv("../train_preliminary/ad.csv")
train_click_log=pd.read_csv("../train_preliminary/click_log.csv")
train_user=pd.read_csv("../train_preliminary/user.csv")

In [11]:
train_ad

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213
...,...,...,...,...,...,...
2481130,4445714,3812196,9152,3,56976,248
2481131,4445715,3812197,39063,17,427,\N
2481132,4445716,3812198,39063,17,427,\N
2481133,4445717,3812199,35885,17,427,\N


In [12]:
train_click_log

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1
...,...,...,...,...
30082766,6,30920,228129,1
30082767,57,30920,2361397,1
30082768,26,30920,629802,1
30082769,86,30920,2713031,1


In [13]:
train_user

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1
...,...,...,...
899995,899996,5,1
899996,899997,3,2
899997,899998,4,2
899998,899999,3,1


In [14]:
train_merged=pd.merge(train_ad,train_click_log,on='creative_id')
train_merged=pd.merge(train_merged,train_user,on='user_id')

In [15]:
# train_merged.describe()

In [16]:
train_merged["product_id"].replace({"\\N":"0"},inplace=True)
train_merged["industry"].replace({"\\N":"0"},inplace=True)
train_merged=train_merged.astype(int)
display(train_merged)

,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times,age,gender
0,1,1,0,5,381,78,81,398695,1,4,1
1,68590,63392,0,18,14681,88,14,398695,1,4,1
2,127337,114621,0,18,9409,25,3,398695,1,4,1
3,138157,124124,23,2,9065,319,8,398695,1,4,1
4,147066,132027,0,18,22918,319,1,398695,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
30082766,4263153,3657208,1810,2,61922,319,90,630049,1,3,2
30082767,4291152,3680507,2400,2,57709,246,89,630049,1,3,2
30082768,4374249,3750607,0,18,10986,6,89,630049,2,3,2
30082769,4376574,3752641,1261,2,10988,6,90,630049,2,3,2


In [17]:
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]
targets=["age","gender"]

In [18]:
tmpdf=train_merged.sample(frac=0.1)
tmpdf.info()
display(tmpdf)
tmpdf.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3008277 entries, 21921163 to 2885313
Data columns (total 11 columns):
 #   Column            Dtype
---  ------            -----
 0   creative_id       int64
 1   ad_id             int64
 2   product_id        int64
 3   product_category  int64
 4   advertiser_id     int64
 5   industry          int64
 6   time              int64
 7   user_id           int64
 8   click_times       int64
 9   age               int64
 10  gender            int64
dtypes: int64(11)
memory usage: 275.4 MB


,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times,age,gender
21921163,2796709,2404215,23,2,18965,319,73,689065,1,3,1
23531778,4097183,3516142,0,18,24894,319,84,700399,1,3,1
12086773,1849865,1601598,3315,2,47236,238,42,588497,1,1,1
5107216,35391,34286,129,2,22123,6,3,569306,1,3,1
4294160,2297097,1981671,1261,2,19056,6,67,192353,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...
27206057,3321332,2857491,43761,17,55259,0,89,292974,1,6,1
11967643,3877658,3331375,37284,4,1535,0,82,756974,1,4,1
28719380,1167126,1022056,0,18,14681,6,26,635860,1,7,2
10435939,294913,264922,8859,3,2608,247,3,223759,1,9,1


,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times,age,gender
count,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06,3.008277e+06
mean,1.676166e+06,1.447605e+06,3.934647e+03,8.126673e+00,2.119303e+04,1.611514e+02,4.800546e+01,4.498864e+05,1.062906e+00,4.232725e+00,1.316769e+00
std,1.308048e+06,1.120568e+06,9.130057e+03,7.054799e+00,1.406013e+04,1.290596e+02,2.592557e+01,2.599259e+05,2.976958e-01,2.049800e+00,4.652167e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,4.158100e+05,3.711100e+05,0.000000e+00,2.000000e+00,1.098600e+04,8.000000e+00,2.600000e+01,2.246580e+05,1.000000e+00,3.000000e+00,1.000000e+00
50%,1.508165e+06,1.310317e+06,1.290000e+02,3.000000e+00,1.810300e+04,2.030000e+02,4.900000e+01,4.499330e+05,1.000000e+00,4.000000e+00,1.000000e+00
75%,2.739708e+06,2.354861e+06,1.873000e+03,1.800000e+01,3.003400e+04,2.930000e+02,7.100000e+01,6.752710e+05,1.000000e+00,6.000000e+00,2.000000e+00
max,4.445715e+06,3.812197e+06,4.431200e+04,1.800000e+01,6.296500e+04,3.350000e+02,9.100000e+01,9.000000e+05,3.600000e+01,1.000000e+01,2.000000e+00


In [20]:
# !pip install lightgbm
%reset -f
import lightgbm as lgb
from sklearn.model_selection import train_test_split
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]

df=train_merged.drop(columns=["age","user_id"])
# convert {1,2} to binary value {0,1}
df["gender"]-=1
# split train data into 2 parts (cross validation)
train=df.sample(frac=0.8) #random state is a seed value
test=df.drop(train.index)
y_train=train.pop("gender")
X_train=train
y_test=test.pop("gender")
X_test=test
# display(X_train,y_train)
# display(X_test,y_test)
# train_data.save_binary('test.bin')
params = {
    'boosting_type': 'gbdt', 
    'objective': 'binary', 
    'metric':'auc',
    'learning_rate': 0.1, 
    'num_leaves': 255, 
    'max_depth': 9,
    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
    'min_data_in_leaf':0,
    'nthread':10,
    }
train_data = lgb.Dataset(X_train,y_train)
cv_results = lgb.cv(
    params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)
print('best num_boost_round:', len(cv_results['auc-mean']))
print('best cv score:', cv_results['auc-mean'])
# bst = lgb.train(param, train_data, num_round,categorical_feature=features)
# bst.save_model('test_model.txt')

NameError: name 'train_merged' is not defined

In [ ]:
# !pip install lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]

df=tmpdf.drop(columns=["gender","user_id"])
# split train data into 2 parts (cross validation)
train=df.sample(frac=0.8) #random state is a seed value
test=df.drop(train.index)
y_train=train.pop("age")
X_train=train
y_test=test.pop("age")
X_test=test
# display(X_train,y_train)
# display(X_test,y_test)
# train_data.save_binary('test.bin')

params = {
    'boosting_type': 'gbdt', 
    'objective': 'regression', 
    'metric':'rmse',
    'learning_rate': 0.1, 
    'num_leaves': 90, 
    'max_depth': 6,
    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
    'min_data_in_leaf':5,
    'nthread':12,
    }

train_data = lgb.Dataset(X_train,y_train)
cv_results = lgb.cv(
    params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)
print('best num_boost_round:', len(cv_results['rmse-mean']))
print('best cv score:', cv_results['rmse-mean'])
# bst = lgb.train(param, train_data, num_round,categorical_feature=features)
# bst.save_model('test_model.txt')

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'nthread':12,
          'learning_rate':0.1
          }
 
### 交叉验证(调参)
print('交叉验证')
max_auc = float('0')
best_params = {}
 
# 准确率
print("调参1：提高准确率")
for num_leaves in range(5,100,5):
    for max_depth in range(3,8,1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
 
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
            
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
            
        if mean_auc >= max_auc:
            max_auc = mean_auc
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
if 'num_leaves' and 'max_depth' in best_params.keys():          
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']
 
# 过拟合
print("调参2：降低过拟合")
for max_bin in range(5,256,10):
    for min_data_in_leaf in range(1,102,10):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=1,
                                nfold=5,
                                metrics=['auc'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
                    
            mean_auc = pd.Series(cv_results['auc-mean']).max()
            boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
            if mean_auc >= max_auc:
                max_auc = mean_auc
                best_params['max_bin']= max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf
if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']
    params['max_bin'] = best_params['max_bin']
 
print("调参3：降低过拟合")
for feature_fraction in [0.6,0.7,0.8,0.9,1.0]:
    for bagging_fraction in [0.6,0.7,0.8,0.9,1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=1,
                                nfold=5,
                                metrics=['auc'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
                    
            mean_auc = pd.Series(cv_results['auc-mean']).max()
            boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
            if mean_auc >= max_auc:
                max_auc=mean_auc
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq
 
if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']
 
 
print("调参4：降低过拟合")
for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
                
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
        if mean_auc >= max_auc:
            max_auc=mean_auc
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']
 
print("调参5：降低过拟合2")
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                        params,
                        lgb_train,
                        seed=1,
                        nfold=5,
                        metrics=['auc'],
                        early_stopping_rounds=10,
                        verbose_eval=True
                        )
            
    mean_auc = pd.Series(cv_results['auc-mean']).max()
    boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
    if mean_auc >= max_auc:
        max_auc=mean_auc
        
        best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
 
print(best_params)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df=tmpdf
features=["creative_id"]
v = TfidfVectorizer(max_df=1.0,min_df=1,lowercase=False,tokenizer=lambda x:x)
for feature in features:
    cur=df.groupby("user_id")[feature].apply(list)
    v_fit = v.fit_transform(cur)
    word=v.get_feature_names()
    display(v_fit)
    display(type(v_fit))
#     for i in range(len(weight)):
#         print("-----output feature={}, user_id={} tf-idf-----".format(feature,i+1))
#         for j in range(len(word)):
#             print(word[j],weight[i][j])